# Source Catalog

Author: Bingcheng Jin

Date: 2023.12.31


## Table of Contents
* [Merge Images](#merge-images)
* [Source Detection](#source-detection)
  * [convolve](#convolve-image-with-guassian-kernel)
  * [apply detection](#apply-source-detection-and-deblend-with-photutils)
* [Assign SEGMAP](#assign-segmentationimage-to-merged-image)
* [Match](#match-sources)
* [Human check](#human-check)

In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [2]:
img_name = 'nircam8'
img_dir = f'./io/image/{img_name}/'
img_list = glob.glob(img_dir + 'hlsp*.fits')
img_list.sort()

## Merge Images

In [3]:
from utils.merge import merge_image

In [4]:
img_list

['./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f115w_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f150w_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f200w_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f277w_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f356w_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f410m_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f444w_dr0.6_i2d.fits']

In [5]:
from astropy.wcs import WCS
merged_dir = f'./io/image/{img_name}/'
image_merged_path = merged_dir+'merged.fits'
if os.path.exists(image_merged_path):
    print('image_merged.fits already exists')
    with fits.open(image_merged_path) as hdul:
        image_merged = hdul['MERGED'].data
        w = WCS(hdul['MERGED'].header)
    print(f'Data are loaded from {image_merged_path}')
else:
    image_merged = merge_image(img_list[3:7])
    w = WCS(fits.getheader(img_list[1], extname='SCI'))
    hdr = w.to_header()
    hdr['EXTNAME'] = 'MERGED'
    fits.writeto(image_merged_path, image_merged, overwrite=True, header=hdr)        

merging with:
./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f277w_dr0.6_i2d.fits
./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f356w_dr0.6_i2d.fits
./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f410m_dr0.6_i2d.fits
./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f444w_dr0.6_i2d.fits


Set DATE-AVG to '2022-12-21T21:34:22.806' from MJD-AVG.
Set DATE-END to '2022-12-21T22:01:15.165' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to     8.975355 from OBSGEO-[XYZ].
Set OBSGEO-H to 1677786988.005 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


## Source Detection

In [6]:
from astropy.stats import sigma_clipped_stats, gaussian_fwhm_to_sigma
from astropy.convolution import Gaussian2DKernel, convolve
from photutils.segmentation import detect_sources

### determine threshold for detection

In [7]:
mean, median, std = sigma_clipped_stats(image_merged, sigma=3.0)
std

0.0045943745498245765

### convolve image with guassian kernel

In [8]:
# might take 40 seconds
fwhm = 4.0 # approaximate pixel FWHM of the NIRCam LW imaging PSF
sigma = fwhm * gaussian_fwhm_to_sigma # FWHM to sigma conversion
image_merged_conv = convolve(image_merged, Gaussian2DKernel(sigma))

### apply source detection and deblend with photutils

In [9]:
segm = detect_sources(image_merged_conv, threshold=1.0*std, npixels=5)

In [10]:
# might take 2-3 minutes
from photutils.segmentation import deblend_sources
segm_deblend = deblend_sources(image_merged_conv, segm, npixels=5)

Deblending:   0%|          | 0/56794 [00:00<?, ?it/s]

## Assign SegmentationImage to merged image

total number of sources detected

In [11]:
segm_deblend.nlabels

94300

convolved image and segmentation map can be used later without doing steps above

In [12]:
hdr_segm = fits.Header()
hdr_segm['EXTNAME'] = 'SEGMAP'
hdr_conv = fits.Header()
hdr_conv['EXTNAME'] = 'SCI_BKSUB_CONV'
fits.append(image_merged_path, image_merged_conv, header=hdr_conv, overwrite=True)
fits.append(image_merged_path, segm_deblend.data, header=hdr_segm, overwrite=True)

## Match Sources

In [13]:
from photutils.segmentation import SourceCatalog, SegmentationImage
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS

In [14]:
with fits.open(f'io/image/{img_name}/merged.fits') as hdul:
    w = WCS(hdul[0].header)
    image_merged_conv = hdul['SCI_BKSUB_CONV'].data
    segm_deblend = SegmentationImage(hdul['SEGMAP'].data)

Set OBSGEO-B to     8.975355 from OBSGEO-[XYZ].
Set OBSGEO-H to 1677786988.005 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
